# TP2 - Organización de Datos
#### Notebook principal

<hr>

### Notebooks utilizados:

- ***pre_processing:*** notebook para el manejo inicial de los dataframes.
- ***feature_generation:*** primer etapa del pipeline. En este notebook se generarán nuevos features para luego, realizar un proceso de selección de los mejores features para cada modelo.
- ***feature_selection*** segunda etapa, donde se buscara encontrar los features con mayor importancia, es decir aquellos que aporten mayor informacion.
- ***parameter_tuning:*** tercer etapa, notebook donde se tunean los parámetros para cada modelo.
- ***predict:*** finalmente, una vez obtenidos los mejores parametros y features para cada modelo, este notebook se encargará de generar el csv con las predicciones finales para el modelo que se le indique.

<hr>


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 7

In [2]:
import nbimporter

from pre_processing import load_featured_datasets
import feature_generation
import feature_selection
import parameter_tuning
import predict

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from feature_selection.ipynb
Importing Jupyter notebook from parameter_tuning.ipynb
Importing Jupyter notebook from predict.ipynb


In [3]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

<hr>

# Resultados obtenidos

# area de testing:

In [2]:
# ...

### Modelo: Regresion lineal

In [4]:
# ...

### Modelo: Regresion logistica

In [5]:
# ...

### Modelo: SVM

In [6]:
# ...

### Modelo: Decision Tree

In [7]:
# ...

### Modelo: RandomForest

In [4]:
from sklearn.ensemble import RandomForestRegressor

In [5]:
train,test = load_featured_datasets()

In [6]:
train.shape

(240000, 137)

In [7]:
train.fillna(train.mean(), inplace = True)

In [8]:
train.shape

(240000, 137)

In [10]:
X = train.drop('precio', axis=1).values
Y = train['precio'].values
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [11]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = seed, verbose=2, max_depth=10) 
regressor.fit(X_train, Y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  5.1min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=7, verbose=2,
                      warm_start=False)

In [12]:
from sklearn import metrics

In [15]:
y_pred = regressor.predict(X_val)
print('MAE: ', int(metrics.mean_absolute_error(Y_val, y_pred)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  688548


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


In [16]:
y_pred2 = regressor.predict(X_train)
print('MAE: ', int(metrics.mean_absolute_error(Y_train, y_pred2)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


MAE:  663067


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished


In [20]:
names = train.columns.to_list()
print(sorted(zip(map(lambda x: round(x, 4), regressor.feature_importances_), names), reverse=True))

[(0.4881, 'metroscubiertos'), (0.2571, 'ciudad_le'), (0.0352, 'ciudad_muycara'), (0.0317, 'banos'), (0.0158, 'tipodepropiedad_1_pol'), (0.0141, 'dia'), (0.0129, 'precio_promedio_metrocubierto_mes'), (0.0125, 'antiguedad'), (0.0113, 'garages'), (0.0105, 'servicio'), (0.0096, 'es_Veracruz'), (0.0093, 'metroscubiertos_mean'), (0.009, 'precio'), (0.0085, 'intercept_pol'), (0.0069, 'tipodepropiedad_2_pol'), (0.0065, 'tipodepropiedad_0_pol'), (0.005, 'habitaciones'), (0.0042, 'aniomes'), (0.0033, 'tipodepropiedad_3_pol'), (0.0031, 'ciudad_barata'), (0.0025, 'es_apart'), (0.0024, 'tipodepropiedad_4_pol'), (0.002, 'tipodepropiedad_le'), (0.002, 'ciudad_cara'), (0.0019, 'tipodepropiedad_8_ohe'), (0.0017, 'lujo'), (0.0017, 'aniomes_scaled'), (0.0015, 'mes'), (0.0015, 'es_casa'), (0.0014, 'tipodepropiedad_7_pol'), (0.0014, 'hab_binning_1_ohe'), (0.0013, 'provincia_10_ohe'), (0.0013, 'gimnasio'), (0.0012, 'parrilla'), (0.0011, 'piscina'), (0.0011, 'es_Distrito Federal'), (0.001, 'hab_binning_7_ohe

### Modelo: XGBoost

_Generacion del dataset de train con sus features_

In [4]:
import xgboost
from sklearn.model_selection import GridSearchCV

In [5]:
train,test = load_featured_datasets()

In [6]:
features = feature_generation.get_features()

In [7]:
train_selected = train[["habitaciones","metroscubiertos","antiguedad","garages","banos","gimnasio","piscina","usosmultiples", "centroscomercialescercanos", "escuelascercanas"]\
                       +features["metros"][1]\
                       +features["metros"][2]\
                       +features["tipodepropiedad"][5]\
                       +features["provincia"][2]\
                       +features["ciudad"][4]\
                       +features["fecha"][5]\
                       +features["descripcion"][0]\
                       +features["metricas"][0]\
                       +features["antiguedad"][0]\
                       +features["habitaciones"][0]\
                       +features["idzona"][0]
                       +["precio"]]

In [8]:
test_selected = test[["habitaciones","metroscubiertos","antiguedad","garages","banos","gimnasio","piscina","usosmultiples", "centroscomercialescercanos", "escuelascercanas"]\
                       +features["metros"][1]\
                       +features["metros"][2]\
                       +features["tipodepropiedad"][5]\
                       +features["provincia"][2]\
                       +features["ciudad"][4]\
                       +features["fecha"][5]\
                       +features["descripcion"][0]\
                       +features["metricas"][0]\
                       +features["antiguedad"][0]\
                       +features["habitaciones"][0]\
                       +features["idzona"][0]
                    ]

In [9]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [10]:
X.shape

(240000, 65)

In [487]:
#X_purged = X[:,feature_indices]

In [10]:
#X_train, X_val, Y_train, Y_val = train_test_split(X_purged, Y, test_size=0.2)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)

In [11]:
parametros = {
    'max_depth':[11,12,13,14,15],
    'n_estimators':[100,110,120,130,140],
    'learning_rate': [0.05,0.08,0.1,0.15,0.2,0.3],
    'subsample':[0.5,0.8,0.9,0.7],
    'min_child_weight':[5,10,15,20,30]
}

In [11]:
reg = xgboost.XGBRegressor(max_depth=14,n_estimators=140 ,learning_rate=0.08, verbosity=2,subsample=0.9, min_child_weight=10, n_jobs=4)
reg.fit(X,Y)

[15:47:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 5404 extra nodes, 0 pruned nodes, max_depth=14
[15:47:40] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 5874 extra nodes, 0 pruned nodes, max_depth=14
[15:47:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6286 extra nodes, 0 pruned nodes, max_depth=14
[15:47:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6596 extra nodes, 0 pruned nodes, max_depth=14
[15:47:43] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6976 extra nodes, 0 pruned nodes, max_depth=14
[15:47:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 7516 extra nodes, 0 pruned nodes, max_depth=14
[15:47:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.08, max_delta_step=0,
             max_depth=14, min_child_weight=10, missing=None, n_estimators=140,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=2)

_Comprobacion contra el conjunto de validacion_

In [37]:
Y_pred = reg.predict(X_val)
mean_absolute_error(Y_val,Y_pred)

510411.868680013

In [32]:
# preparamos el csv de respuesta para kaggle

In [12]:
ids = test_selected.index.values
X_test = test_selected.values


In [13]:
test_predict = reg.predict(X_test)

In [14]:
escribir_respuesta(ids, test_predict)

### Modelo: CatBoost

In [ ]:
#...

### Modelo: LightGBM

In [4]:
import lightgbm as lgb

In [32]:
train,test = load_featured_datasets()

In [33]:
features = feature_generation.get_features()

In [34]:
best_features = feature_selection.get_best_features_per_category()

In [35]:
best_features

[('metros', 1),
 ('tipodepropiedad', 0),
 ('provincia', 6),
 ('ciudad', 2),
 ('fecha', 4),
 ('descripcion', 0),
 ('metricas', 2),
 ('habitaciones', 0),
 ('antiguedad', 1),
 ('extras', 2),
 ('volcanes', 0),
 ('idzona', 0)]

In [36]:
train_selected = train[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                        'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                        'centroscomercialescercanos']\
                       +features["metros"][1]\
                       +features["tipodepropiedad"][0]\
                       +features["provincia"][6]\
                       +features["ciudad"][2]\
                       +features["fecha"][4]\
                       +features["descripcion"][0]\
                       +features["metricas"][2]\
                       +features["habitaciones"][0]\
                       +features["antiguedad"][1]\
                       +features["extras"][2]\
                       +features["volcanes"][0]\
                       +features["idzona"][0]\
                       +["precio"]]

test_selected = test[['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                        'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
                        'centroscomercialescercanos']\
                       +features["metros"][1]\
                       +features["tipodepropiedad"][0]\
                       +features["provincia"][6]\
                       +features["ciudad"][2]\
                       +features["fecha"][4]\
                       +features["descripcion"][0]\
                       +features["metricas"][2]\
                       +features["habitaciones"][0]\
                       +features["antiguedad"][1]\
                       +features["extras"][2]\
                       +features["volcanes"][0]\
                       +features["idzona"][0]]

In [41]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [42]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.05, random_state=seed)

In [43]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 200}
n_estimators=20000

In [44]:
d_train = lgb.Dataset(X_train, label=Y_train)
d_valid = lgb.Dataset(X_val, label=Y_val)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

[1]	valid_0's l1: 1.5419e+06
Training until validation scores don't improve for 200 rounds
[2]	valid_0's l1: 1.48596e+06
[3]	valid_0's l1: 1.43356e+06
[4]	valid_0's l1: 1.38389e+06
[5]	valid_0's l1: 1.33726e+06
[6]	valid_0's l1: 1.29341e+06
[7]	valid_0's l1: 1.252e+06
[8]	valid_0's l1: 1.21369e+06
[9]	valid_0's l1: 1.17663e+06
[10]	valid_0's l1: 1.14227e+06
[11]	valid_0's l1: 1.11033e+06
[12]	valid_0's l1: 1.07971e+06
[13]	valid_0's l1: 1.05118e+06
[14]	valid_0's l1: 1.0244e+06
[15]	valid_0's l1: 999323
[16]	valid_0's l1: 975260
[17]	valid_0's l1: 952676
[18]	valid_0's l1: 932143
[19]	valid_0's l1: 912588
[20]	valid_0's l1: 894495
[21]	valid_0's l1: 877164
[22]	valid_0's l1: 861436
[23]	valid_0's l1: 846106
[24]	valid_0's l1: 831702
[25]	valid_0's l1: 817755
[26]	valid_0's l1: 805643
[27]	valid_0's l1: 793043
[28]	valid_0's l1: 781866
[29]	valid_0's l1: 771045
[30]	valid_0's l1: 760339
[31]	valid_0's l1: 751122
[32]	valid_0's l1: 741375
[33]	valid_0's l1: 732948
[34]	valid_0's l1: 7250

[305]	valid_0's l1: 518149
[306]	valid_0's l1: 518096
[307]	valid_0's l1: 518037
[308]	valid_0's l1: 517852
[309]	valid_0's l1: 517804
[310]	valid_0's l1: 517769
[311]	valid_0's l1: 517659
[312]	valid_0's l1: 517591
[313]	valid_0's l1: 517505
[314]	valid_0's l1: 517480
[315]	valid_0's l1: 517423
[316]	valid_0's l1: 517414
[317]	valid_0's l1: 517250
[318]	valid_0's l1: 517202
[319]	valid_0's l1: 517175
[320]	valid_0's l1: 517180
[321]	valid_0's l1: 517115
[322]	valid_0's l1: 517089
[323]	valid_0's l1: 517052
[324]	valid_0's l1: 516949
[325]	valid_0's l1: 516925
[326]	valid_0's l1: 516872
[327]	valid_0's l1: 516821
[328]	valid_0's l1: 516778
[329]	valid_0's l1: 516688
[330]	valid_0's l1: 516536
[331]	valid_0's l1: 516462
[332]	valid_0's l1: 516437
[333]	valid_0's l1: 516337
[334]	valid_0's l1: 516280
[335]	valid_0's l1: 516259
[336]	valid_0's l1: 516175
[337]	valid_0's l1: 516142
[338]	valid_0's l1: 516018
[339]	valid_0's l1: 515945
[340]	valid_0's l1: 515891
[341]	valid_0's l1: 515845
[

[618]	valid_0's l1: 504546
[619]	valid_0's l1: 504530
[620]	valid_0's l1: 504488
[621]	valid_0's l1: 504451
[622]	valid_0's l1: 504431
[623]	valid_0's l1: 504383
[624]	valid_0's l1: 504319
[625]	valid_0's l1: 504303
[626]	valid_0's l1: 504283
[627]	valid_0's l1: 504197
[628]	valid_0's l1: 504162
[629]	valid_0's l1: 504144
[630]	valid_0's l1: 504111
[631]	valid_0's l1: 504095
[632]	valid_0's l1: 504040
[633]	valid_0's l1: 503938
[634]	valid_0's l1: 503904
[635]	valid_0's l1: 503849
[636]	valid_0's l1: 503818
[637]	valid_0's l1: 503822
[638]	valid_0's l1: 503801
[639]	valid_0's l1: 503782
[640]	valid_0's l1: 503790
[641]	valid_0's l1: 503739
[642]	valid_0's l1: 503731
[643]	valid_0's l1: 503688
[644]	valid_0's l1: 503618
[645]	valid_0's l1: 503594
[646]	valid_0's l1: 503560
[647]	valid_0's l1: 503558
[648]	valid_0's l1: 503529
[649]	valid_0's l1: 503464
[650]	valid_0's l1: 503479
[651]	valid_0's l1: 503477
[652]	valid_0's l1: 503450
[653]	valid_0's l1: 503381
[654]	valid_0's l1: 503355
[

[927]	valid_0's l1: 496518
[928]	valid_0's l1: 496499
[929]	valid_0's l1: 496478
[930]	valid_0's l1: 496413
[931]	valid_0's l1: 496424
[932]	valid_0's l1: 496411
[933]	valid_0's l1: 496382
[934]	valid_0's l1: 496367
[935]	valid_0's l1: 496317
[936]	valid_0's l1: 496329
[937]	valid_0's l1: 496313
[938]	valid_0's l1: 496292
[939]	valid_0's l1: 496207
[940]	valid_0's l1: 496177
[941]	valid_0's l1: 496125
[942]	valid_0's l1: 496103
[943]	valid_0's l1: 496088
[944]	valid_0's l1: 496058
[945]	valid_0's l1: 496050
[946]	valid_0's l1: 496047
[947]	valid_0's l1: 496039
[948]	valid_0's l1: 496030
[949]	valid_0's l1: 495992
[950]	valid_0's l1: 496000
[951]	valid_0's l1: 495935
[952]	valid_0's l1: 495908
[953]	valid_0's l1: 495843
[954]	valid_0's l1: 495793
[955]	valid_0's l1: 495747
[956]	valid_0's l1: 495739
[957]	valid_0's l1: 495685
[958]	valid_0's l1: 495681
[959]	valid_0's l1: 495628
[960]	valid_0's l1: 495619
[961]	valid_0's l1: 495597
[962]	valid_0's l1: 495592
[963]	valid_0's l1: 495565
[

[1225]	valid_0's l1: 490188
[1226]	valid_0's l1: 490179
[1227]	valid_0's l1: 490170
[1228]	valid_0's l1: 490158
[1229]	valid_0's l1: 490134
[1230]	valid_0's l1: 490115
[1231]	valid_0's l1: 490092
[1232]	valid_0's l1: 490087
[1233]	valid_0's l1: 490105
[1234]	valid_0's l1: 490057
[1235]	valid_0's l1: 490016
[1236]	valid_0's l1: 489985
[1237]	valid_0's l1: 489982
[1238]	valid_0's l1: 490001
[1239]	valid_0's l1: 490001
[1240]	valid_0's l1: 490002
[1241]	valid_0's l1: 489960
[1242]	valid_0's l1: 489949
[1243]	valid_0's l1: 489896
[1244]	valid_0's l1: 489879
[1245]	valid_0's l1: 489866
[1246]	valid_0's l1: 489837
[1247]	valid_0's l1: 489809
[1248]	valid_0's l1: 489781
[1249]	valid_0's l1: 489757
[1250]	valid_0's l1: 489740
[1251]	valid_0's l1: 489727
[1252]	valid_0's l1: 489692
[1253]	valid_0's l1: 489706
[1254]	valid_0's l1: 489680
[1255]	valid_0's l1: 489670
[1256]	valid_0's l1: 489655
[1257]	valid_0's l1: 489654
[1258]	valid_0's l1: 489617
[1259]	valid_0's l1: 489641
[1260]	valid_0's l1:

...

[15066]	valid_0's l1: 453562
[15067]	valid_0's l1: 453565
[15068]	valid_0's l1: 453556
[15069]	valid_0's l1: 453550
[15070]	valid_0's l1: 453550
[15071]	valid_0's l1: 453554
[15072]	valid_0's l1: 453559
[15073]	valid_0's l1: 453563
[15074]	valid_0's l1: 453568
[15075]	valid_0's l1: 453569
[15076]	valid_0's l1: 453568
[15077]	valid_0's l1: 453571
[15078]	valid_0's l1: 453575
[15079]	valid_0's l1: 453564
[15080]	valid_0's l1: 453562
[15081]	valid_0's l1: 453560
[15082]	valid_0's l1: 453558
[15083]	valid_0's l1: 453562
[15084]	valid_0's l1: 453573
[15085]	valid_0's l1: 453569
[15086]	valid_0's l1: 453570
[15087]	valid_0's l1: 453584
[15088]	valid_0's l1: 453584
[15089]	valid_0's l1: 453582
[15090]	valid_0's l1: 453583
[15091]	valid_0's l1: 453581
[15092]	valid_0's l1: 453576
[15093]	valid_0's l1: 453569
[15094]	valid_0's l1: 453574
[15095]	valid_0's l1: 453575
[15096]	valid_0's l1: 453578
[15097]	valid_0's l1: 453572
[15098]	valid_0's l1: 453574
[15099]	valid_0's l1: 453577
[15100]	valid_

[15349]	valid_0's l1: 453447
[15350]	valid_0's l1: 453445
[15351]	valid_0's l1: 453451
[15352]	valid_0's l1: 453447
[15353]	valid_0's l1: 453449
[15354]	valid_0's l1: 453446
[15355]	valid_0's l1: 453441
[15356]	valid_0's l1: 453444
[15357]	valid_0's l1: 453442
[15358]	valid_0's l1: 453440
[15359]	valid_0's l1: 453433
[15360]	valid_0's l1: 453424
[15361]	valid_0's l1: 453417
[15362]	valid_0's l1: 453418
[15363]	valid_0's l1: 453404
[15364]	valid_0's l1: 453402
[15365]	valid_0's l1: 453387
[15366]	valid_0's l1: 453390
[15367]	valid_0's l1: 453389
[15368]	valid_0's l1: 453393
[15369]	valid_0's l1: 453396
[15370]	valid_0's l1: 453391
[15371]	valid_0's l1: 453389
[15372]	valid_0's l1: 453387
[15373]	valid_0's l1: 453391
[15374]	valid_0's l1: 453392
[15375]	valid_0's l1: 453405
[15376]	valid_0's l1: 453411
[15377]	valid_0's l1: 453403
[15378]	valid_0's l1: 453400
[15379]	valid_0's l1: 453400
[15380]	valid_0's l1: 453396
[15381]	valid_0's l1: 453397
[15382]	valid_0's l1: 453405
[15383]	valid_

[15638]	valid_0's l1: 453237
[15639]	valid_0's l1: 453237
[15640]	valid_0's l1: 453237
[15641]	valid_0's l1: 453233
[15642]	valid_0's l1: 453238
[15643]	valid_0's l1: 453232
[15644]	valid_0's l1: 453233
[15645]	valid_0's l1: 453230
[15646]	valid_0's l1: 453225
[15647]	valid_0's l1: 453223
[15648]	valid_0's l1: 453224
[15649]	valid_0's l1: 453223
[15650]	valid_0's l1: 453228
[15651]	valid_0's l1: 453232
[15652]	valid_0's l1: 453227
[15653]	valid_0's l1: 453219
[15654]	valid_0's l1: 453213
[15655]	valid_0's l1: 453212
[15656]	valid_0's l1: 453213
[15657]	valid_0's l1: 453218
[15658]	valid_0's l1: 453218
[15659]	valid_0's l1: 453219
[15660]	valid_0's l1: 453219
[15661]	valid_0's l1: 453221
[15662]	valid_0's l1: 453225
[15663]	valid_0's l1: 453220
[15664]	valid_0's l1: 453214
[15665]	valid_0's l1: 453216
[15666]	valid_0's l1: 453216
[15667]	valid_0's l1: 453215
[15668]	valid_0's l1: 453212
[15669]	valid_0's l1: 453210
[15670]	valid_0's l1: 453208
[15671]	valid_0's l1: 453205
[15672]	valid_

[15921]	valid_0's l1: 453095
[15922]	valid_0's l1: 453085
[15923]	valid_0's l1: 453083
[15924]	valid_0's l1: 453074
[15925]	valid_0's l1: 453065
[15926]	valid_0's l1: 453058
[15927]	valid_0's l1: 453058
[15928]	valid_0's l1: 453057
[15929]	valid_0's l1: 453054
[15930]	valid_0's l1: 453059
[15931]	valid_0's l1: 453059
[15932]	valid_0's l1: 453061
[15933]	valid_0's l1: 453064
[15934]	valid_0's l1: 453063
[15935]	valid_0's l1: 453060
[15936]	valid_0's l1: 453064
[15937]	valid_0's l1: 453057
[15938]	valid_0's l1: 453066
[15939]	valid_0's l1: 453056
[15940]	valid_0's l1: 453056
[15941]	valid_0's l1: 453052
[15942]	valid_0's l1: 453052
[15943]	valid_0's l1: 453058
[15944]	valid_0's l1: 453053
[15945]	valid_0's l1: 453046
[15946]	valid_0's l1: 453046
[15947]	valid_0's l1: 453045
[15948]	valid_0's l1: 453041
[15949]	valid_0's l1: 453045
[15950]	valid_0's l1: 453047
[15951]	valid_0's l1: 453047
[15952]	valid_0's l1: 453047
[15953]	valid_0's l1: 453049
[15954]	valid_0's l1: 453045
[15955]	valid_

[16211]	valid_0's l1: 452988
[16212]	valid_0's l1: 452991
[16213]	valid_0's l1: 452987
[16214]	valid_0's l1: 452989
[16215]	valid_0's l1: 452985
[16216]	valid_0's l1: 452993
[16217]	valid_0's l1: 452988
[16218]	valid_0's l1: 452987
[16219]	valid_0's l1: 452984
[16220]	valid_0's l1: 452982
[16221]	valid_0's l1: 452980
[16222]	valid_0's l1: 452979
[16223]	valid_0's l1: 452982
[16224]	valid_0's l1: 452980
[16225]	valid_0's l1: 452989
[16226]	valid_0's l1: 452983
[16227]	valid_0's l1: 452977
[16228]	valid_0's l1: 452979
[16229]	valid_0's l1: 452976
[16230]	valid_0's l1: 452977
[16231]	valid_0's l1: 452978
[16232]	valid_0's l1: 452976
[16233]	valid_0's l1: 452979
[16234]	valid_0's l1: 452981
[16235]	valid_0's l1: 452985
[16236]	valid_0's l1: 452988
[16237]	valid_0's l1: 452988
[16238]	valid_0's l1: 452990
[16239]	valid_0's l1: 452991
[16240]	valid_0's l1: 452992
[16241]	valid_0's l1: 452990
[16242]	valid_0's l1: 452993
[16243]	valid_0's l1: 452991
[16244]	valid_0's l1: 452991
[16245]	valid_

In [22]:
Y_pred = reg.predict(X_val)
mean_absolute_error(Y_val,Y_pred)

487268.28239609225

In [331]:
# preparamos el csv de respuesta para kaggle

In [46]:
ids = test_selected.index.values
X_test = test_selected.values

In [47]:
test_predict = reg.predict(X_test)
escribir_respuesta(ids, test_predict)

In [ ]:
# best params so far
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 50}
n_estimators=10000

### Modelo: KNN

### Modelo: Neural Networks

In [3]:
# ...